In [1]:
import copy
import os
import re
import time
import json
import math
from tqdm import tqdm
import argparse
from datasets import load_dataset
import itertools
import numpy as np

from retrievers import calculate_retrieval_metrics

/weka/nora-default/varshak/pyenv/versions/miniconda3-3.9-24.1.2-0/envs/bright/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
raw_examples = load_dataset('xlangai/bright', f"examples", cache_dir="cache")["theoremqa_theorems"]
examples = {}
for e in raw_examples:
    examples[e['id']] = e

raw_examples1 = load_dataset('xlangai/bright', f"gpt4_reason", cache_dir="cache")["theoremqa_theorems"]
examples1 = {}
for e in raw_examples1:
    examples1[e['id']] = e

In [ ]:
li1 = [ex['id'] for ex in examples.values()]
li2 = [ex['id'] for ex in examples1.values()]

In [21]:
len(li1), len(li2)

(76, 78)

In [2]:
raw_examples = load_dataset('xlangai/bright', f"gpt4_reason", cache_dir="cache")["economics"]
examples = {}
for e in raw_examples:
    examples[e['id']] = e

doc_pairs = load_dataset('xlangai/bright', 'documents', cache_dir="cache")["economics"]
documents = {}
for d in doc_pairs:
    documents[d['id']] = d['content']

In [3]:
with open("outputs/economics_bm25_long_False_reasoning_gpt4/score.json") as f:
    all_scores = json.load(f)
new_scores = copy.deepcopy(all_scores)

with open("r1_score_5x_0_rank_r_results.jsonl", "r") as f:
    data_0 = [json.loads(line) for line in f]

with open("r1_score_5x_1_rank_r_results.jsonl", "r") as f:
    data_1 = [json.loads(line) for line in f]

with open("r1_score_5x_2_rank_r_results.jsonl", "r") as f:
    data_2 = [json.loads(line) for line in f]

with open("r1_score_5x_3_rank_r_results.jsonl", "r") as f:
    data_3 = [json.loads(line) for line in f]

with open("r1_score_5x_4_rank_r_results.jsonl", "r") as f:
    data_4 = [json.loads(line) for line in f]

with open("llama_score_5x_0_rank_r_results.jsonl", "r") as f:
    data_b = [json.loads(line) for line in f]

In [13]:
for i in range(len(data)):
    # print disagreements in data and datab
    da = ((np.asanyarray(data[i]['scores']) != np.asanyarray(data_b[i]['scores'])).sum())
    # print(da)
    if da>30:
        print(i, da)

10 33
15 32
23 38
27 32
39 39
43 34
52 34
53 41
59 33
61 36
74 33
92 31
94 49
95 31


In [19]:
# extract doc order
dids = []
for qid,scores in tqdm(all_scores.items()):
    docs = []
    sorted_scores = sorted(scores.items(),key=lambda x:x[1],reverse=True)[:100]
    for did, _ in sorted_scores:
        docs.append([did, documents[did]])
    dids.append([did for did, _ in sorted_scores])

100%|██████████| 103/103 [00:00<00:00, 4612.18it/s]


In [82]:
ind = 21
# dids[ind].index("dollar_flow/Circularflowofincome_16.txt")
dids[ind][34]

'interest_rate_parity/eurron_13.txt'

In [46]:
# (data[ind]['scores']), (data_b[ind]['scores'])
data[ind]['scores'][72]

0.0

In [63]:
print(data[ind]['scores'])
print(np.where(np.asarray(data[ind]['scores']) == 1.0)[0])


[0.0, 0.2, 0.6, 0.4, 0.2, 0.4, 0.2, 0.2, 0.8, 0.4, 0.4, 0.0, 0.0, 0.4, 0.4, 0.2, 0.4, 0.2, 0.2, 0.2, 0.4, 0.2, 0.4, 0.4, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.0, 0.2, 0.4, 0.4, 0.2, 0.2, 0.2, 0.4, 0.0, 0.0, 0.0, 0.4, 0.2, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.2, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.4, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.4, 0.4, 0.2, 0.0, 0.0, 0.0]
[]


In [137]:
print(data[ind]['outputs'][12])

Relevance score: 3
<end_answer>
</think><think>
Okay, so I need to determine if the provided document is relevant to the query about GDP calculation and whether it can be negative, especially in a simplified economy where a toy is produced and sold at a loss. Let me break this down step by step.

First, the query is about understanding GDP calculation, specifically whether GDP can be negative. The example given involves producing a toy in one year and selling it at a loss in the next, and how that affects GDP. The query also touches on inventory changes and their impact on GDP components like investment and consumption.

Looking at the document, it's from a chapter titled "Change in Private Inventories" and discusses how the Bureau of Economic Analysis (BEA) adjusts inventory data. It goes into detail about accounting methods, price indexes, and how inventories are valued. The document mentions the Inventory Valuation Adjustment (IVA) and how it's used to remove holding gains or losses

In [83]:
print(documents["interest_rate_parity/eurron_13.txt"])

forecasted period might be determined
by the future evolution of several macroeconomic factors such as: the increase
of the current account deficit correlated
with the potential increase in the public
budget deficit (the twin deficits), the ascending trend of the Romanian interest
rates on the money market, the upward
trend of inflation in Romania, or the expansionary fiscal policy which has begun
in 2016 as well as by changing regulations
and political instability.
It worth’s to be mentioned that recently the National Bank of Romania [15]
has launched the national plan for euro
adoption in order to map the future path
to be followed by decision makers in this
regard, arguing that it is only a matter
of timing for passing to euro. It is ascertained the complexity of this process and
the careful assessment of the full accomplishment of all convergence criteria so
as to witness a smooth euro adoption.
SWS INTERNATIONAL
SOCIETY
SWS Journal of Social Sciences and Art
- 41 - DOI 10.35603/SS

In [ ]:
# for combing with BM25
for qid,scores in tqdm(all_scores.items()):
    for i, output in enumerate(data[int(qid)]['outputs']):
        data[int(qid)]['ranking'][dids[int(qid)][i]] = data[int(qid)]['ranking'][dids[int(qid)][i]]*100 + scores[dids[int(qid)][i]]

for qid,scores in tqdm(all_scores.items()):
    new_scores[qid] = data[int(qid)]['ranking']

In [80]:
scores_5x = []
for i in range(5):
    with open(f"r1_score_5x_{i}_rank_r_results.jsonl", "r") as f:
        data = [json.loads(line) for line in f]
        scores_5x.append(data[21]['scores'])
scores_5x = np.asarray(scores_5x)
np.where(np.any(scores_5x > 0.6, axis=0) & np.any(scores_5x < 0.4, axis=0))[0]

array([34])

In [121]:
scores_5x = np.asarray(scores_5x)
# find the column with the highest variance
np.var(scores_5x, axis=0)


array([0.0224, 0.0064, 0.032 , 0.0096, 0.0096, 0.0224, 0.0096, 0.    ,
       0.0064, 0.    , 0.0096, 0.0064, 0.0064, 0.0064, 0.0096, 0.0096,
       0.    , 0.0096, 0.0224, 0.0064, 0.    , 0.0096, 0.    , 0.0064,
       0.    , 0.0064, 0.    , 0.    , 0.0096, 0.    , 0.    , 0.0064,
       0.    , 0.    , 0.    , 0.0096, 0.0096, 0.    , 0.    , 0.0096,
       0.0096, 0.0064, 0.    , 0.    , 0.    , 0.    , 0.    , 0.0064,
       0.    , 0.    , 0.0064, 0.    , 0.    , 0.    , 0.0064, 0.    ,
       0.0064, 0.0064, 0.0064, 0.    , 0.    , 0.0096, 0.    , 0.0384,
       0.    , 0.0064, 0.016 , 0.    , 0.0064, 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.0256, 0.    , 0.0096, 0.    ,
       0.0096, 0.    , 0.    , 0.    , 0.0096, 0.0096, 0.    , 0.0064,
       0.    , 0.    , 0.0096, 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.0096, 0.    , 0.    ])

In [81]:
scores_5x[:, 34] 

array([0. , 0.8, 0.2, 0.2, 0. ])

In [124]:
# I want to find a column that has a value >0.5 and <0.5
np.where((scores_5x[:, 1] > 0.5) & (scores_5x[:, 1] < 0.6))

(array([], dtype=int64),)

In [84]:
print(data_1[ind]['outputs'][34])

Okay, so I need to figure out if the provided document is relevant to the query. The query is about why there aren't many studies on forecasting exchange rates of Central Bank Digital Currencies (CBDCs) compared to cryptocurrencies. It goes into details about the nature of CBDCs, their pegged exchange rates, and reasons why research might be lacking.

Looking at the document, it's about forecasting the RON/EUR exchange rate. It discusses macroeconomic factors like current account deficits, interest rates, inflation, and fiscal policies. It also mentions the National Bank of Romania's plan to adopt the euro and references several studies on exchange rate modeling using various econometric models like ARIMA, GARCH, etc.

So, the document is focused on traditional fiat currency exchange rates, specifically RON to EUR. It doesn't mention CBDCs at all. The query is about the lack of studies on CBDC exchange rate forecasting. Since CBDCs are pegged to their fiat currencies, their exchange ra

In [85]:
print(data_2[ind]['outputs'][34])

Okay, so I need to determine if the provided document is relevant to the query. The query is about why there's a lack of studies on forecasting exchange rates for Central Bank Digital Currencies (CBDCs), especially compared to cryptocurrencies. The document, on the other hand, seems to discuss exchange rate forecasting models, specifically for the RON/EUR rate, using econometric models like ARIMA, GARCH, etc.

First, the query is focused on CBDCs and why their exchange rate forecasting isn't studied as much. The document doesn't mention CBDCs at all. It talks about traditional fiat currencies, like the Romanian Lei (RON) and the Euro (EUR). So, the subject matter is different.

Next, the query explores reasons such as the pegged nature of CBDCs, their developmental stage, and regulatory aspects. The document, however, is about applying various forecasting models to existing fiat currencies. It doesn't touch on digital currencies, let alone CBDCs.

The query also mentions that CBDC exch